## Propertime Quickstart

Propertime is an attempt at proper time management in Python.

In a nutshell, it provides two main classes: ``Time`` for representing time (similar to a datetime) and ``TimeUnit`` for representing units of time (similar to timedelta). 

Such classes are implement assuming two strict and opinionated base hypotheses:

- Time is a floating point number representing the number of seconds passed after the zero on the time axis, which is set to 1st January 1970 UTC, any other representations (as date/hours, time zones, daylight saving times) are built on top of it, and:

- Time units can be of both fixed and *variable* length, if defined with calendar time units as days, weeks, months and years. This means that the length (i.e. the duration in seconds) of a one-day time unit is not defined *unless* it it put in context, which means to know on which time it is applied.

These two assumptions allows Propertime to solve by design many issues in manipulating time that are still present in Python's built-in datetime module as well as in most third-party libraries.

Propertime provides a simple and neat API, and its objects play nice with Python datetimes so that you can mix and match and use it only when needed. Implementing "proper" time comes indeed at a price: it optimizes for consistency over performance and it is quite strict, meaning that its suitability heavily depends on the use-case.

This is the quickstart notebook. You might also want to check out the [reference documentation](https://propertime.readthedocs.io).

### The Time class

*NOTE: in order to use Propertime's time arithmetic features you don't need to use the Time class, you can jump straight to the TimeUnit class below.*

The Time class is how Propertime represents time.

In [1]:
from propertime import Time

To get the time right now, just create a new Time instance without any arguments:

In [2]:
Time()

Time: 1703974824.0 (2023-12-30 22:20:24 UTC)

The main concept in Propertime is that time is always expressed as Epoch seconds (from 1st January 1970 UTC). Then, it can be represented in different ways or on different time zones and with different offsets. But at its core, a Time objects is a floating point number, and can always be casted as such:

In [3]:
float(Time())

1703974824.0

To get the current time on another timezone or using an offset, use the `tz` of `offset` arguments:

In [4]:
Time(tz='Europe/Rome')

Time: 1703974824.0 (2023-12-30 23:20:24 Europe/Rome)

In [5]:
Time(offset=-72000)

Time: 1703974824.0 (2023-12-30 02:20:24 -20:00)

To instead create a Time instance at a given time, either use Epoch seconds, classic datetime-like arguments, a (ISO) string representation or a datetime object. Naive strings or datetimes without an extra time zone or UTC offset are always assumed on UTC, as Propertime does not allow for naive Time instances.

In [6]:
Time(1703520720.0)

Time: 1703520720.0 (2023-12-25 16:12:00 UTC)

In [7]:
Time(2023,12,3,16,12,0)

Time: 1701619920.0 (2023-12-03 16:12:00 UTC)

In [8]:
Time('2023-12-25T16:12:00+01:00', tz='Europe/Rome')

Time: 1703517120.0 (2023-12-25 16:12:00 Europe/Rome)

In [9]:
from datetime import datetime
Time(datetime(2023,12,3,16,12,0))

Time: 1701619920.0 (2023-12-03 16:12:00 UTC)

Once a Time object has been initialized, it behaves exactly as a floting point number. A quick example to compute the average of an arrival time:

In [10]:
arrival_times = [Time(datetime(2023,12,3,16,12,0)), Time(datetime(2023,12,3,16,56,0)), Time(datetime(2023,12,3,16,3,0))]
Time(sum(arrival_times)/len(arrival_times))

Time: 1701620620.0 (2023-12-03 16:23:40 UTC)

... and it can be converted back and forth to datetimes or (ISO) string representations:

In [11]:
time = Time(datetime(2023,12,3,16,12,0))

In [12]:
time.dt()

datetime.datetime(2023, 12, 3, 16, 12, tzinfo=<UTC>)

In [13]:
time.iso()

'2023-12-03T16:12:00+00:00'

### The TimeUnit class
Time units impement precise, calendar-awarare time arithmetic and can hopefully reduce the headaches caused by time manipulation operations. Their main charachteristic is to embarace time units with *variable* length (e.g a day can last 23, 24 or 25 hours) and to correctly handle all the various edge cases.

Morover, TimeUnits' arithmetic also accepts that some operations can be not well defined (e.g. adding a month to the 31st of January) and that should raise an error, exactly as a divison by zero would.

TimeUnits can be instantiated by setting their length in seconds, by manually setting all their components (as for Python datetime objets) or using their string representation: `1s` for one second, `1m` for one minute, `1h` for one hour, `1D` for one day, `1M` for one month, `1W` for one weelk and `1Y` for one year (Physical time units are lowercase, calendar time units are uppercase). Other values other than one are of course supported.


In [14]:
from propertime import TimeUnit
TimeUnit('1h')

1h

In [15]:
assert TimeUnit('1h') == TimeUnit('3600s') == TimeUnit(3600) == TimeUnit(hours=1)

Composite time units are possible as well:

In [16]:
TimeUnit('1h_30m')

1h_30m

Time units with fixed lenght can always get as seconds:

In [17]:
TimeUnit('1h_30m').as_seconds()

5400.0

However, variable lenght time units cannot be quantified in terms of seconds if not contextualised at a given time and ona given timezone: how a one-day time unit last depends on the day, and must therefore know it. The `start` argument servers for this puropose: to "fix" a variable lenght time units thus allowing to get the duration as seconds. It can be either a Time od datetime object.

In [18]:
TimeUnit('1D').as_seconds(start=Time(2023, 3, 26, 0, 0, tz='Europe/Rome'))

82800.0

TimeUnits can indeed be of two main types:

 - "physical", which have an always fixed legth (as seconds, minutes and hours)
 - "calendar", which have a variable lenght depending on a calendar (as days, weeks, months, and years).

This abstraction allows to clearly state that a (fixed) 24-hours time unit is different than a one-day time unit (that can last 23, 24 or 25 hours):

In [19]:
assert TimeUnit('24h') == TimeUnit('86400s') 
assert TimeUnit('1D') != TimeUnit('86400s')

To get the type of a time unit just use the `type` attirbute:

In [20]:
TimeUnit('24h').type

'Physical'

In [21]:
TimeUnit('1D').type

'Calendar'

Time units can be added and subtracted each other and to Time and datetime objects, in which case their context is automatically defined, thus allowing to maniplate time in a consistent way:

In [22]:
Time(2023,1,1,0,0,0) + TimeUnit('1M')

Time: 1675209600.0 (2023-02-01 00:00:00 UTC)

Time units can also be used to round, ceil or floor time.

In [23]:
TimeUnit('1D').round(Time(2023,10,29,16,0,0))

Time: 1698624000.0 (2023-10-30 00:00:00 UTC)

In [24]:
TimeUnit('1D').floor(Time(2023,1,31,19,21,47))

Time: 1675123200.0 (2023-01-31 00:00:00 UTC)

In [25]:
TimeUnit('1D').shift(Time(2023,1,31,19,21,47))

Time: 1675279307.0 (2023-02-01 19:21:47 UTC)

This is very useful for "traveling" around. For example, to get to the noon of the next day, given any point in time:

In [26]:
time = Time(2023,10,29,0,15,39, tz='Europe/Rome')
TimeUnit('1D').ceil(time) + TimeUnit('12h')

Time: 1698663600.0 (2023-10-30 12:00:00 Europe/Rome)

TimeUnits are also useful for "slotting" time, while taking care about all the DST extra complications. For example, to slot a day in 1-hour bins: 

In [27]:
start = Time(2023,10,29,0,0,0, tz='Europe/Rome')
end = start + TimeUnit('1D')

slot_strart_time = start
while slot_strart_time < end:
    print(slot_strart_time)
    slot_strart_time = slot_strart_time + TimeUnit('1h')

Time: 1698530400.0 (2023-10-29 00:00:00 Europe/Rome DST)
Time: 1698534000.0 (2023-10-29 01:00:00 Europe/Rome DST)
Time: 1698537600.0 (2023-10-29 02:00:00 Europe/Rome DST)
Time: 1698541200.0 (2023-10-29 02:00:00 Europe/Rome)
Time: 1698544800.0 (2023-10-29 03:00:00 Europe/Rome)
Time: 1698548400.0 (2023-10-29 04:00:00 Europe/Rome)
Time: 1698552000.0 (2023-10-29 05:00:00 Europe/Rome)
Time: 1698555600.0 (2023-10-29 06:00:00 Europe/Rome)
Time: 1698559200.0 (2023-10-29 07:00:00 Europe/Rome)
Time: 1698562800.0 (2023-10-29 08:00:00 Europe/Rome)
Time: 1698566400.0 (2023-10-29 09:00:00 Europe/Rome)
Time: 1698570000.0 (2023-10-29 10:00:00 Europe/Rome)
Time: 1698573600.0 (2023-10-29 11:00:00 Europe/Rome)
Time: 1698577200.0 (2023-10-29 12:00:00 Europe/Rome)
Time: 1698580800.0 (2023-10-29 13:00:00 Europe/Rome)
Time: 1698584400.0 (2023-10-29 14:00:00 Europe/Rome)
Time: 1698588000.0 (2023-10-29 15:00:00 Europe/Rome)
Time: 1698591600.0 (2023-10-29 16:00:00 Europe/Rome)
Time: 1698595200.0 (2023-10-29 17:

Note that the above "travels" happened across DST changes, which were correcly handled in both cases.

### Consistency and edge cases

One of the main features of Propertime is to enforce consistency at differente levels and to handle quite well several edge cases that are usually not handled by Python built-in time management utilities or other third-party libraries.

Let's for example see what happens if we try to add a month to the 31st of january, which is an operation not well defined:

In [28]:
try:
    Time(2023,1,31,0,0,0) + TimeUnit('1M')
except Exception as e:
    print(e)

Day is out of range for month for 2023-01-31 00:00:00+00:00 plus 1 month(s)


Similarly, let's see if we add one day to the 2:15 AM *before* a DST change when skipping an hour:

In [29]:
try:
    Time(2023,3,25,2,15,0, tz='Europe/Rome') + TimeUnit('1D')
except Exception as e:
    print(e)

Cannot shift "2023-03-25 02:15:00+01:00" by "1D" (Sorry, time 2023-03-26 02:15:00 does not exist on time zone Europe/Rome)


This happens because 2:15 AM on March 26th, 2023 actually does not exists on the `Europe/Rome` time zone. And neither 2:15 AM on March 12th, 2023 on `America/New_York` does:

In [30]:
try:
    Time(2023,3,12,2,15,0, tz='America/New_York')
except Exception as e:
    print(e)

Sorry, time 2023-03-12 02:15:00 does not exist on time zone America/New_York


A similar logic applies when the DST adjustment goes in the other way, by duplicating an hour:

In [31]:
try:
    Time(2023,10,28,2,15,0, tz='Europe/Rome')+ TimeUnit('1D')
except Exception as e:
    print(e)

Cannot shift "2023-10-28 02:15:00+02:00" by "1D" (Would end up on time 2023-10-29 02:15:00 which is ambiguous on time zone Europe/Rome)


This is because 2:15 AM on October 28, 2023 actually represent two points in time on time zone `Europe/Rome`: before and after the DST kicking in. Exacly as for 1:15 AM on the 5t of November, 2023, on time zone `America/New_York`:

In [32]:
try:
    Time(2023,11,5,1,15,0, tz='America/New_York')
except Exception as e:
    print(e)

Sorry, time 2023-11-05 01:15:00 is ambiguous on time zone America/New_York without an offset


You can force creating such time enabling the "guessing" mode, but it will only be possible to create one of the two:

In [33]:
Time(2023,11,5,1,15,0, tz='America/New_York', can_guess=True)

Time 2023-11-05 01:15:00 is ambiguous on time zone America/New_York, assuming -05:00 UTC offset


Time: 1699164900.0 (2023-11-05 01:15:00 America/New_York)

To get the other one, either set the offset (explicitly or with a string conversion, it's the same):

In [34]:
assert Time(2023,11,5,1,15,0, offset=-3600*4, tz='America/New_York') == Time('2023-11-05T01:15:00-04:00', tz='America/New_York')

In [35]:
Time(2023,11,5,1,15,0, offset=-3600*4, tz='America/New_York')

Time: 1699161300.0 (2023-11-05 01:15:00 America/New_York DST)

...or just add the necessary hours to a previous point in time, which is an operation always well defined:

In [36]:
Time(2023,11,5,0,15,0, tz='America/New_York') + TimeUnit('1h')

Time: 1699161300.0 (2023-11-05 01:15:00 America/New_York DST)

In [37]:
Time(2023,11,5,0,15,0, tz='America/New_York') + TimeUnit('2h')

Time: 1699164900.0 (2023-11-05 01:15:00 America/New_York)

### Where to go from here

You can check out the [reference documentation](https://propertime.readthedocs.io), or you can contribute!